In [2]:

%pip install biopython transformers torch datasets numpy scikit-learn evaluate

%pip install transformers[torch]



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# %%
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
import numpy as np
import os
from Bio import SeqIO
import evaluate
import inspect

print("✅ Imports complete")


✅ Imports complete


In [4]:
# %%
# Clear GPU cache before starting
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✅ GPU cache cleared")
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Total GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   Available memory: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected.")


✅ GPU cache cleared
   GPU: NVIDIA GeForce RTX 4090
   Total GPU memory: 25.25 GB
   Available memory: 25.25 GB


In [5]:
import sys
sys.path.append("/home/mluser/AFML_RISHABH/progen")

In [6]:
from progen.progen2.models.progen.modeling_progen import ProGenForCausalLM


In [7]:
import sys
import torch
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers import PreTrainedTokenizerFast
from progen.progen2.models.progen.modeling_progen import ProGenForCausalLM

# Add your repo to sys.path
sys.path.append("/home/mluser/AFML_RISHABH/progen/progen2")


In [8]:
model_path = "/home/mluser/AFML_RISHABH/progen2-small"
tokenizer_path = "/home/mluser/AFML_RISHABH/progen/progen2/tokenizer.json"

tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_path)
model = ProGenForCausalLM.from_pretrained(model_path)


ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. 

In [9]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/home/mluser/AFML_RISHABH/progen2-small")

# If pad_token is missing, assign it to eos_token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

print(f"PAD TOKEN: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")


PAD TOKEN: </s>, ID: 30


In [11]:
print("✅ Model and tokenizer loaded successfully!")
print("Vocab size:", tokenizer.vocab_size)


✅ Model and tokenizer loaded successfully!
Vocab size: 30


In [12]:
# %%
class FASTADataset(Dataset):
    def __init__(self, fasta_file, tokenizer, max_length=512):
        print(f"Loading sequences from {fasta_file}...")
        self.sequences = []
        for record in SeqIO.parse(fasta_file, "fasta"):
            seq = str(record.seq)
            if len(seq) > 0:
                self.sequences.append(seq)
        
        self.tokenizer = tokenizer
        self.max_length = max_length
        print(f"Loaded {len(self.sequences)} sequences")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        spaced_seq = " ".join(list(seq))
        
        encoding = self.tokenizer(
            spaced_seq,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        result = {k: v.squeeze(0) for k, v in encoding.items()}
        
        if 'attention_mask' not in result:
            result['attention_mask'] = torch.ones_like(result['input_ids'])
        
        return result

print("✅ FASTADataset class defined")


✅ FASTADataset class defined


In [13]:
# %%
data_folder = '/home/mluser/AFML_RISHABH/10k sequences'

train_path = os.path.join(data_folder, "kinases_cluster_train_10k.fasta")
val_path   = os.path.join(data_folder, "kinases_cluster_val_10k.fasta")
test_path  = os.path.join(data_folder, "kinases_cluster_test_10k.fasta")

# Verify files exist
for path in [train_path, val_path, test_path]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    print(f"✓ Found: {path}")

MAX_LENGTH = 512

train_dataset = FASTADataset(train_path, tokenizer, max_length=MAX_LENGTH)
val_dataset   = FASTADataset(val_path, tokenizer, max_length=MAX_LENGTH)
test_dataset  = FASTADataset(test_path, tokenizer, max_length=MAX_LENGTH)

print("\n" + "="*50)
print(f"Dataset sizes: {len(train_dataset)}, {len(val_dataset)}, {len(test_dataset)}")
print(f"Max sequence length: {MAX_LENGTH}")
print("="*50)


✓ Found: /home/mluser/AFML_RISHABH/10k sequences/kinases_cluster_train_10k.fasta
✓ Found: /home/mluser/AFML_RISHABH/10k sequences/kinases_cluster_val_10k.fasta
✓ Found: /home/mluser/AFML_RISHABH/10k sequences/kinases_cluster_test_10k.fasta
Loading sequences from /home/mluser/AFML_RISHABH/10k sequences/kinases_cluster_train_10k.fasta...
Loaded 7989 sequences
Loading sequences from /home/mluser/AFML_RISHABH/10k sequences/kinases_cluster_val_10k.fasta...
Loaded 1002 sequences
Loading sequences from /home/mluser/AFML_RISHABH/10k sequences/kinases_cluster_test_10k.fasta...
Loaded 1009 sequences

Dataset sizes: 7989, 1002, 1009
Max sequence length: 512


In [14]:
# %%# Force FP32 forward pass and handle unexpected kwargs
def force_fp32_forward(original_forward):
    sig = inspect.signature(original_forward)
    
    def wrapped_forward(*args, **kwargs):
        valid_params = set(sig.parameters.keys())
        filtered_kwargs = {k: v for k, v in kwargs.items() if k in valid_params}
        with torch.autocast(device_type='cuda', enabled=False):
            return original_forward(*args, **filtered_kwargs)
    return wrapped_forward

model.forward = force_fp32_forward(model.forward)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device).to(torch.float32)
torch.set_float32_matmul_precision("high")

if hasattr(model, "gradient_checkpointing_enable"):
    try:
        model.gradient_checkpointing_enable()
        print("✓ Gradient checkpointing enabled")
    except ValueError:
        print("⚠️ Gradient checkpointing not supported for this model — continuing without it")
else:
    print("⚠️ gradient_checkpointing_enable() not found — continuing without it")

print(f"✅ Model ready on {device}")

if torch.cuda.is_available():
    print(f"   GPU memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"   GPU memory reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")


print(f"✅ Model ready on {device}")

if torch.cuda.is_available():
    print(f"   GPU memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"   GPU memory reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")


⚠️ Gradient checkpointing not supported for this model — continuing without it
✅ Model ready on cuda
   GPU memory allocated: 0.62 GB
   GPU memory reserved: 0.62 GB
✅ Model ready on cuda
   GPU memory allocated: 0.62 GB
   GPU memory reserved: 0.62 GB


In [15]:
# %%
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [16]:
# %%
from itertools import product

learning_rates = [3e-5, 5e-5, 7e-5]
weight_decays = [0.01, 0.05]
batch_sizes = [1, 2]
gradient_accumulation_steps_list = [4, 8]
max_seq_lengths = [512]
num_train_epochs = 10

hyperparameter_combinations = list(product(
    learning_rates,
    weight_decays,
    batch_sizes,
    gradient_accumulation_steps_list,
    max_seq_lengths
))

print(f"Total combinations to run: {len(hyperparameter_combinations)}")


Total combinations to run: 24


In [17]:
# %%
def get_training_args(run_id, learning_rate, weight_decay, batch_size, grad_accum, max_seq_len, output_dir=None):
    if output_dir is None:
        output_dir = f"./ProGen_10k_hparam_run_{run_id}"

    return TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum,
        save_strategy="epoch",
        save_total_limit=num_train_epochs,
        logging_steps=100,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        fp16=False,
        bf16=False,
        eval_strategy="no",
        dataloader_num_workers=0,
        remove_unused_columns=False,
        gradient_checkpointing=False,
        optim="adamw_torch",
        max_grad_norm=1.0,
        logging_first_step=True,
        report_to="none",
    )

print("✅ TrainingArguments factory ready")


✅ TrainingArguments factory ready


In [18]:
from contextlib import nullcontext
from accelerate.state import AcceleratorState
import gc, os


try:
    if hasattr(AcceleratorState, "_shared_state") and AcceleratorState._shared_state:
        AcceleratorState._reset_state()
        print("✓ Cleared accelerator state")
except Exception as e:
    print(f"Note: Could not clear accelerator state: {e}")

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("✓ Cleared GPU cache")

training_args = get_training_args(
    run_id=0,
    learning_rate=5e-5,
    weight_decay=0.01,
    batch_size=1,
    grad_accum=8,
    max_seq_len=512
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.autocast_smart_context_manager = nullcontext

print("✅ Trainer created successfully (no evaluation during training)")


✓ Cleared GPU cache
✅ Trainer created successfully (no evaluation during training)


/tmp/ipykernel_24263/249755329.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
# %%
import gc
gc.collect()
torch.cuda.empty_cache()


In [19]:
# %%
import torch, gc, pandas as pd, os
from contextlib import nullcontext
from accelerate.state import AcceleratorState

BASE_DIR = "ProGen_hparam_runs"
os.makedirs(BASE_DIR, exist_ok=True)
MASTER_CSV = os.path.join(BASE_DIR, "summary.csv")

results = []
completed = set()

if os.path.exists(MASTER_CSV):
    prev_df = pd.read_csv(MASTER_CSV)
    for _, row in prev_df.iterrows():
        combo = (row["learning_rate"], row["weight_decay"], row["batch_size"], row["grad_accum"], row["max_seq_len"])
        completed.add(combo)
    results = prev_df.to_dict(orient="records")
    print(f"🔁 Loaded {len(completed)} completed runs from {MASTER_CSV}")
else:
    print("🆕 Starting fresh — no previous runs found.")

for i, (lr, wd, bs, grad_acc, max_len) in enumerate(hyperparameter_combinations, 1):
    combo = (lr, wd, bs, grad_acc, max_len)
    if combo in completed:
        print(f"⏭️ Skipping Run {i}: already done (lr={lr}, wd={wd}, bs={bs}, grad_acc={grad_acc}, len={max_len})")
        continue

    run_name = f"run_{i:03d}_lr{lr}_wd{wd}_bs{bs}_ga{grad_acc}_len{max_len}"
    run_dir = os.path.join(BASE_DIR, run_name)
    os.makedirs(run_dir, exist_ok=True)

    print("\n" + "="*100)
    print(f"🏁 Starting Run {i}/{len(hyperparameter_combinations)} → {run_name}")
    print("="*100)

    try:
        if hasattr(AcceleratorState, "_shared_state") and AcceleratorState._shared_state:
            AcceleratorState._reset_state()
            print("✓ Cleared accelerator state")
    except Exception as e:
        print(f"⚠️ Could not clear accelerator state: {e}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("✓ Cleared GPU cache")

    training_args = get_training_args(
        run_id=i,
        learning_rate=lr,
        weight_decay=wd,
        batch_size=bs,
        grad_accum=grad_acc,
        max_seq_len=max_len,
        output_dir=run_dir
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    trainer.autocast_smart_context_manager = nullcontext

    try:
        train_result = trainer.train()
        status = "Success"
        print(f"✅ Run {i} completed successfully!")

        trainer.save_model(run_dir)
        tokenizer.save_pretrained(run_dir)
        torch.save(training_args, os.path.join(run_dir, "training_args.pt"))

        if hasattr(trainer.state, "log_history"):
            pd.DataFrame(trainer.state.log_history).to_csv(os.path.join(run_dir, "log_history.csv"), index=False)
    except RuntimeError as e:
        status = "OOM" if "out of memory" in str(e).lower() else "Failed"
    except Exception as e:
        status = f"Error: {str(e)[:80]}"
    finally:
        results.append({
            "run_id": i,
            "learning_rate": lr,
            "weight_decay": wd,
            "batch_size": bs,
            "grad_accum": grad_acc,
            "max_seq_len": max_len,
            "output_dir": run_dir,
            "status": status,
        })
        completed.add(combo)
        pd.DataFrame(results).to_csv(MASTER_CSV, index=False)

        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()
        print("🧹 CUDA memory cleared.\n")

print("\n✅ All hyperparameter runs completed!")
print(f"📦 Master summary available at: {MASTER_CSV}")


/tmp/ipykernel_22216/1171570660.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 30, 'bos_token_id': None, 'pad_token_id': 30}.


🔁 Loaded 22 completed runs from ProGen_hparam_runs/summary.csv
⏭️ Skipping Run 1: already done (lr=3e-05, wd=0.01, bs=1, grad_acc=4, len=512)
⏭️ Skipping Run 2: already done (lr=3e-05, wd=0.01, bs=1, grad_acc=8, len=512)
⏭️ Skipping Run 3: already done (lr=3e-05, wd=0.01, bs=2, grad_acc=4, len=512)
⏭️ Skipping Run 4: already done (lr=3e-05, wd=0.01, bs=2, grad_acc=8, len=512)
⏭️ Skipping Run 5: already done (lr=3e-05, wd=0.05, bs=1, grad_acc=4, len=512)
⏭️ Skipping Run 6: already done (lr=3e-05, wd=0.05, bs=1, grad_acc=8, len=512)
⏭️ Skipping Run 7: already done (lr=3e-05, wd=0.05, bs=2, grad_acc=4, len=512)
⏭️ Skipping Run 8: already done (lr=3e-05, wd=0.05, bs=2, grad_acc=8, len=512)
⏭️ Skipping Run 9: already done (lr=5e-05, wd=0.01, bs=1, grad_acc=4, len=512)
⏭️ Skipping Run 10: already done (lr=5e-05, wd=0.01, bs=1, grad_acc=8, len=512)
⏭️ Skipping Run 11: already done (lr=5e-05, wd=0.01, bs=2, grad_acc=4, len=512)
⏭️ Skipping Run 12: already done (lr=5e-05, wd=0.01, bs=2, grad_ac

Step,Training Loss
1,12.410100
100,8.168400
200,7.493200
300,6.986400
400,6.919000
500,6.790200
600,6.696400
700,6.545900
800,6.427700
900,6.273800


ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. 

✅ Run 23 completed successfully!
🧹 CUDA memory cleared.


🏁 Starting Run 24/24 → run_024_lr7e-05_wd0.05_bs2_ga8_len512
✓ Cleared accelerator state
✓ Cleared GPU cache


/tmp/ipykernel_22216/1171570660.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,0.237000
100,0.748900
200,0.951200
300,1.079200
400,1.155900
500,1.214800
600,0.972000
700,1.041700
800,1.095300
900,1.139200


ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. 

✅ Run 24 completed successfully!
🧹 CUDA memory cleared.


✅ All hyperparameter runs completed!
📦 Master summary available at: ProGen_hparam_runs/summary.csv


In [22]:
!pip install --upgrade transformers

Could not fetch URL https://pypi.org/simple/transformers/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/transformers/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)'))) - skipping


In [19]:
from transformers import AutoConfig, AutoModelForCausalLM
from progen.progen2.models.progen.configuration_progen import ProGenConfig 
# your custom definitions

# Register your custom model so Transformers recognizes it
AutoConfig.register("progen", ProGenConfig)
AutoModelForCausalLM.register(ProGenConfig, ProGenForCausalLM)


In [31]:
import json, os

for run in ["run_007_lr3e-05_wd0.05_bs2_ga4_len512", "run_015_lr5e-05_wd0.05_bs2_ga4_len512"]:
    cfg_path = f"/home/mluser/AFML_RISHABH/ProGen_hparam_runs/{run}/config.json"
    with open(cfg_path) as f:
        cfg = json.load(f)
    print(run, "→ vocab_size =", cfg.get("vocab_size"))


run_007_lr3e-05_wd0.05_bs2_ga4_len512 → vocab_size = 31
run_015_lr5e-05_wd0.05_bs2_ga4_len512 → vocab_size = 32


In [20]:
# %%
import os
import math
import torch
import pandas as pd
from torch.utils.data import ConcatDataset, DataLoader
from tqdm import tqdm
from transformers import AutoModelForCausalLM, PreTrainedTokenizerFast, DataCollatorForLanguageModeling
import warnings

# --- Device setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings("ignore", message=".*GenerationMixin.*")
warnings.filterwarnings("ignore", message=".*not initialized from the model checkpoint.*")
warnings.filterwarnings("ignore", category=UserWarning)

# --- Base directory for all runs ---
base_dir = "/home/mluser/AFML_RISHABH/ProGen_hparam_runs"
results_csv = os.path.join(base_dir, "evaluation_results.csv")

# --- Combine validation and test datasets ---
full_dataset = ConcatDataset([val_dataset, test_dataset])

# --- Safe forward wrapper to handle model errors ---
def safe_forward(forward_fn):
    def wrapper(*args, **kwargs):
        try:
            return forward_fn(*args, **kwargs)
        except Exception as e:
            print("⚠️ Forward pass failed:", e)
            return type("DummyOutput", (), {"loss": None})
    return wrapper


# --- Load existing results (if present) ---
if os.path.exists(results_csv):
    prev_df = pd.read_csv(results_csv)
    completed_runs = set(prev_df["model_run"].dropna())
    results = prev_df.to_dict(orient="records")
    print(f"🔁 Loaded {len(completed_runs)} evaluated runs from {results_csv}")
else:
    completed_runs = set()
    results = []
    print("🆕 Starting fresh evaluation — no previous results found.")


# --- Iterate through all trained runs ---
for run_name in sorted(os.listdir(base_dir)):
    run_path = os.path.join(base_dir, run_name)
    if not os.path.isdir(run_path):
        continue

    # Skip already evaluated runs
    if run_name in completed_runs:
        print(f"⏭️ Skipping {run_name} — already evaluated.")
        continue

    print(f"\n🔍 Evaluating model: {run_name}")

    try:
        # --- Load tokenizer ---
        tokenizer = PreTrainedTokenizerFast.from_pretrained(run_path)

        # Ensure pad_token is set
        if tokenizer.pad_token is None:
            tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

        # --- Load model (trust remote code for custom ProGen architecture) ---
        model = AutoModelForCausalLM.from_pretrained(
            run_path,
            trust_remote_code=True,
            ignore_mismatched_sizes=True
        )
        model.resize_token_embeddings(len(tokenizer))
        model.forward = safe_forward(model.forward)
        model = model.to(device).eval()

        # --- Data collator (recreated for each model to avoid tokenization mismatch) ---
        collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
        loader = DataLoader(full_dataset, batch_size=8, collate_fn=collator)

        total_loss = 0.0
        total_count = 0

        with torch.no_grad():
            for batch in tqdm(loader, desc=f"Evaluating {run_name}", leave=False):
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(**batch)
                if outputs.loss is not None:
                    total_loss += outputs.loss.item() * batch["input_ids"].size(0)
                    total_count += batch["input_ids"].size(0)

        if total_count > 0:
            avg_loss = total_loss / total_count
            ppl = math.exp(avg_loss)
            print(f"✅ {run_name} — Loss: {avg_loss:.4f}, Perplexity: {ppl:.2f}")
        else:
            avg_loss, ppl = None, None
            print(f"⚠️ {run_name} — No valid batches.")

        # --- Save results to list ---
        results.append({
            "model_run": run_name,
            "avg_loss": avg_loss,
            "perplexity": ppl
        })

        # --- Save per-run CSV as well ---
        pd.DataFrame([{
            "model_run": run_name,
            "avg_loss": avg_loss,
            "perplexity": ppl
        }]).to_csv(os.path.join(run_path, "eval_result.csv"), index=False)

    except Exception as e:
        print(f"❌ Error evaluating {run_name}: {e}")
        results.append({"model_run": run_name, "avg_loss": None, "perplexity": None})

    # --- Save progress incrementally ---
    pd.DataFrame(results).to_csv(results_csv, index=False)

# --- Final save ---
pd.DataFrame(results).to_csv(results_csv, index=False)
print(f"\n📦 Evaluation completed — results saved to: {results_csv}")


ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Some weights of ProGenForCausalLM were not initialized from the model checkpoint at /home/mluser/AFML_RISHABH/ProGen_hparam_runs/run_001_lr3e-05_wd0.01_bs1_ga4_len512 and are newly initialized because the shapes did not match:
- lm_head.bias: found shape torch.Size([32]) in the checkpoint and torch.Size([31]) in the model instantiated
- lm_head.weight: found shape torch.Size([32, 1024]) in the checkpoint and torch.Size([31, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the mod

🔁 Loaded 18 evaluated runs from /home/mluser/AFML_RISHABH/ProGen_hparam_runs/evaluation_results.csv

🔍 Evaluating model: run_001_lr3e-05_wd0.01_bs1_ga4_len512


ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of ProGenForCausalLM were not initialized from the model checkpoint at /home/mluser/AFML_RISHABH/ProGen_hparam_runs/run_002_lr3e-05_wd0.01_bs1_ga8_len512

✅ run_001_lr3e-05_wd0.01_bs1_ga4_len512 — Loss: 5.9343, Perplexity: 377.77

🔍 Evaluating model: run_002_lr3e-05_wd0.01_bs1_ga8_len512


...

ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of ProGenForCausalLM were not initialized from the model checkpoint at /home/mluser/AFML_RISHABH/ProGen_hparam_runs/run_005_lr3e-05_wd0.05_bs1_ga4_len512

✅ run_004_lr3e-05_wd0.01_bs2_ga8_len512 — Loss: 5.7352, Perplexity: 309.59

🔍 Evaluating model: run_005_lr3e-05_wd0.05_bs1_ga4_len512


ProGenForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of ProGenForCausalLM were not initialized from the model checkpoint at /home/mluser/AFML_RISHABH/ProGen_hparam_runs/run_006_lr3e-05_wd0.05_bs1_ga8_len512

✅ run_005_lr3e-05_wd0.05_bs1_ga4_len512 — Loss: 5.7135, Perplexity: 302.92

🔍 Evaluating model: run_006_lr3e-05_wd0.05_bs1_ga8_len512


✅ run_006_lr3e-05_wd0.05_bs1_ga8_len512 — Loss: 5.2181, Perplexity: 184.58
⏭️ Skipping run_015_lr5e-05_wd0.05_bs2_ga4_len512 — already evaluated.
⏭️ Skipping run_016_lr5e-05_wd0.05_bs2_ga8_len512 — already evaluated.
⏭️ Skipping run_017_lr7e-05_wd0.01_bs1_ga4_len512 — already evaluated.
⏭️ Skipping run_018_lr7e-05_wd0.01_bs1_ga8_len512 — already evaluated.
⏭️ Skipping run_019_lr7e-05_wd0.01_bs2_ga4_len512 — already evaluated.
⏭️ Skipping run_020_lr7e-05_wd0.01_bs2_ga8_len512 — already evaluated.
⏭️ Skipping run_021_lr7e-05_wd0.05_bs1_ga4_len512 — already evaluated.
⏭️ Skipping run_022_lr7e-05_wd0.05_bs1_ga8_len512 — already evaluated.
⏭️ Skipping run_023_lr7e-05_wd0.05_bs2_ga4_len512 — already evaluated.
⏭️ Skipping run_024_lr7e-05_wd0.05_bs2_ga8_len512 — already evaluated.

📦 Evaluation completed — results saved to: /home/mluser/AFML_RISHABH/ProGen_hparam_runs/evaluation_results.csv
